In [ ]:
class DataSink():
    """
    Abstract Class
    """
    def __init__(self, df, path, method, partition_by = False, params=None):
        self.df = df
        self.path = path
        self.method = method
        self.partition_by = partition_by
        self.params = params

    def load_df(self):
        """
        Abstract method, it will function will be defined in sub classes
        """
        raise NotImplementedError("This method has not been implemented yet.")

In [ ]:
class LoadToDBFS(DataSink):

    def load_df(self):
        if not self.partition_by:
            self.df.write.mode(self.method).save(self.path)
            return

        partition_by_columns = self.params.get(
            "partition_by_columns"
        )
        self.df.write.mode(self.method).partitionBy(*partition_by_columns).save(self.path)

class LoadToDelta(DataSink):
    def load_df(self):
        self.df.write.format("delta").mode(self.method).saveAsTable(self.path)

In [ ]:
sink_sources = {
    "dbfs": LoadToDBFS,
    "delta": LoadToDelta
}

def get_sink_source(df, sink_type, path, method, partition_by=False, params=None):
    sink_source = sink_sources.get(sink_type, None)

    if not sink_source:
        raise NotImplementedError(f"Sink Type `{sink_type}` is not implemented!")

    return sink_source(
        df=df,
        path=path,
        method=method,
        partition_by=partition_by,
        params=params
    )
